# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [54]:
# librerias

import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem

import pylab as plt

import seaborn as sns

from scipy.stats import bernoulli, beta   

from bayes import *

In [55]:
# datos
ab_test = pd.read_csv('data/cookie_cats.csv')

ab_test.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [56]:
# transformacion
ab_test.version.value_counts()      #   No hay errores entre los usuarios de cada versión de puerta

ab_test.retention_1.value_counts()  #   Tampoco en retention 1

ab_test.retention_7.value_counts()  #   Ni en retention 7


retention_7
False    73408
True     16781
Name: count, dtype: int64

### AB Testing 

In [57]:
# Muestreo RETENTION 1

control_ret1 = ab_test[ab_test.version=='gate_30']['retention_1']

trat_ret1 = ab_test[ab_test.version=='gate_40']['retention_1']

In [58]:
# Testeo RETENTION 1

impresiones = [control_ret1.shape[0], trat_ret1.shape[0]]  

conversiones = [sum(control_ret1), sum(trat_ret1)]  



z_score, p_value = proportions_ztest(conversiones, 
                                     nobs=impresiones)


(control_a, trata_a), (control_b, trata_b) = proportion_confint(conversiones, 
                                                                nobs=impresiones,
                                                                alpha=0.05)

In [59]:
print(f'z-score: {z_score:.2f}')

print(f'p-valor: {p_value:.3f}')

print(f'intervalo conf 95% para grupo control: [{control_a:.3f}, {control_b:.3f}]')

print(f'intervalo conf 95% para grupo tratamiento: [{trata_a:.3f}, {trata_b:.3f}]')

z-score: 1.78
p-valor: 0.074
intervalo conf 95% para grupo control: [0.444, 0.453]
intervalo conf 95% para grupo tratamiento: [0.438, 0.447]


Como el pvalor es mayor que 0.05, no podemos rechazar H0, por lo tanto, no existen diferencias significativas entre los usuarios que pagan en la puerta 30 o en la puerta 40 para la retención 1

In [60]:
# Muestreo RETENTION 7

control_ret7 = ab_test[ab_test.version=='gate_30']['retention_7']

trat_ret7 = ab_test[ab_test.version=='gate_40']['retention_7']

In [61]:
# Testeo RETENTION 7

impresiones = [control_ret7.shape[0], trat_ret7.shape[0]]  

conversiones = [sum(control_ret7), sum(trat_ret7)]  



z_score, p_value = proportions_ztest(conversiones, 
                                     nobs=impresiones)


(control_a, trata_a), (control_b, trata_b) = proportion_confint(conversiones, 
                                                                nobs=impresiones,
                                                                alpha=0.05)

In [62]:
print(f'z-score: {z_score:.2f}')

print(f'p-valor: {p_value:.3f}')

print(f'intervalo conf 95% para grupo control: [{control_a:.3f}, {control_b:.3f}]')

print(f'intervalo conf 95% para grupo tratamiento: [{trata_a:.3f}, {trata_b:.3f}]')

z-score: 3.16
p-valor: 0.002
intervalo conf 95% para grupo control: [0.187, 0.194]
intervalo conf 95% para grupo tratamiento: [0.178, 0.186]


In [63]:
#Comparación de medias


### A/B Testing Bayesiano

In [64]:
retention_1= pd.crosstab(ab_test.version, ab_test.retention_1)

retention_1

retention_1,False,True
version,,
gate_30,24666,20034
gate_40,25370,20119


In [65]:
retention_7= pd.crosstab(ab_test.version, ab_test.retention_7)

retention_7

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [66]:
# muestreo

# GATE 30
imp_ctr_1, convs_ctr_1 = 44700, 20034   # entrar, pagar
imp_ctr_7, convs_ctr_7 = 44700, 8502    # entrar, pagar

# GATE 40
imp_trt_1, convs_trt_1 = 45489, 20119   # entrar, pagar
imp_trt_7, convs_trt_7 = 45489, 8279    # entrar, pagar


In [67]:
# TESTEO PARA RETENTION 1

a_C_1, b_C_1 = 1+convs_ctr_1, 1+imp_ctr_1-convs_ctr_1

a_T_1, b_T_1 = 1+convs_trt_1, 1+imp_trt_1-convs_trt_1


beta_control_1 = beta(a_C_1, b_C_1)

beta_test_1 = beta(a_T_1, b_T_1)

beta_control_1.mean(), beta_test_1.mean()

(0.44819023757326293, 0.44228528719966587)

In [68]:
salto_1 = (beta_control_1.mean() - beta_test_1.mean())/beta_control_1.mean()

salto_1

0.013175098158249857

In [69]:
prob_1=calc_prob(beta_test_1, beta_control_1)  

prob_1

0.037206025180329845

In [70]:
print (
'''El test aumenta la tasa de conversion en un {:2.2f}%,
con una probabilidad del {:2.2f}%'''
    .format(salto_1*100, prob_1*100)
)

El test aumenta la tasa de conversion en un 1.32%,
con una probabilidad del 3.72%


In [71]:
# TESTEO PARA RETENTION 7

a_C_7, b_C_7 = 1+convs_ctr_7, 1+imp_ctr_7-convs_ctr_7

a_T_7, b_T_7 = 1+convs_trt_7, 1+imp_trt_7-convs_trt_7

beta_control_7 = beta(a_C_7, b_C_7)

beta_test_7 = beta(a_T_7, b_T_7)

beta_control_7.mean(), beta_test_7.mean()

(0.19021520289919913, 0.18201402475214878)

In [72]:
salto_7 = (beta_control_7.mean() - beta_test_7.mean())/beta_control_7.mean()

salto_7

0.04311526114658881

In [73]:
prob_7=calc_prob(beta_test_7, beta_control_7)  

prob_7

0.0007773386645759051

In [74]:
print (
'''El test aumenta la tasa de conversion en un {:2.2f}%,
con una probabilidad del {:2.2f}%'''
    .format(salto_7*100, prob_7*100)
)

El test aumenta la tasa de conversion en un 4.31%,
con una probabilidad del 0.08%
